workbook for migrating database: filemaker to sqlite

In [1]:
import sqlite3, os
import pandas as pd
import re

In [3]:
#set home directory path
hdir = os.path.expanduser('~')

dh_path = '/Dropbox/Active_Directories/Digital_Humanities/'

database_path = os.path.join(hdir, dh_path.strip('/'), 'database_eurasia_7.0.db')

In [5]:

# Check if database file exists
if not os.path.exists(database_path):
    raise FileNotFoundError(f"Database file not found at: {database_path}")

# Connect to the SQLite database
conn = sqlite3.connect(database_path)

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

try:
    # Your database operations will go here
    pass
finally:
    # Always close the connection when done
    cursor.close()
    conn.close()

In [7]:
# Connect to the SQLite database
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

try:
    # Query to get all table names
    cursor.execute("""
        SELECT name 
        FROM sqlite_master 
        WHERE type='table'
        ORDER BY name;
    """)
    
    # Fetch all results
    tables = cursor.fetchall()
    
    print("Tables in database:")
    for table in tables:
        table_name = table[0]
        # Query to get the number of columns and foreign keys for each table
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns_info = cursor.fetchall()
        num_columns = len(columns_info)

        cursor.execute(f"PRAGMA foreign_key_list({table_name});")
        foreign_keys = cursor.fetchall()
        foreign_keys_info = [fk[3] for fk in foreign_keys]  # Get the foreign key names

        print(f"- Table: {table_name}, Number of Columns: {num_columns}, Foreign Keys: {foreign_keys_info}")

finally:
    cursor.close()
    conn.close()

Tables in database:
- Table: bibliography, Number of Columns: 22, Foreign Keys: ['Repository_ID', 'Author_ID']
- Table: classical_genre, Number of Columns: 7, Foreign Keys: ['Source_ID', 'Knowledge_Form_ID', 'Classical_ID']
- Table: classical_sources, Number of Columns: 20, Foreign Keys: ['Location_ID']
- Table: commentaries, Number of Columns: 5, Foreign Keys: ['Commentating_Work', 'Commentated_Work']
- Table: conquests, Number of Columns: 10, Foreign Keys: ['Defending_Power_ID', 'Conquering_Power_ID', 'Conquered_Territory_ID']
- Table: copies_holdings, Number of Columns: 14, Foreign Keys: ['Scribe_Individual_ID', 'Repository_ID', 'Reference_Source_ID', 'Location_ID', 'Copied_Source_ID', 'Copied_Classical_ID']
- Table: definitions, Number of Columns: 10, Foreign Keys: ['Source_ID', 'Social_Role_ID', 'Lexicon_ID']
- Table: epochs, Number of Columns: 6, Foreign Keys: []
- Table: gazetteer, Number of Columns: 13, Foreign Keys: []
- Table: honorifics, Number of Columns: 4, Foreign Keys: [

In [9]:
def get_unique_values(table_name, column_name):
    """
    Retrieve a list of all unique values in the specified column of a table.
    """
    # Establish a connection to the database using the database_path
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()
    
    # Query to select distinct values from the specified column
    query = f"SELECT DISTINCT {column_name} FROM {table_name};"
    cursor.execute(query)
    
    # Fetch all unique values
    unique_values = [row[0] for row in cursor.fetchall()]
    
    # Close the cursor and connection
    cursor.close()
    conn.close()
    
    return unique_values

In [15]:
#get_unique_values("social_roles", "Type")

In [20]:
def replace_linebreaks_with_comma(table_name, column_name):
    """
    Replace line break delimiters in a specified column with a comma.
    """
    # Establish a connection to the database
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()
    
    # Define the line break delimiters to replace
    linebreaks = ['\x0b', '\n', '\r']
    
    # Create a SQL query to update the column
    for linebreak in linebreaks:
        query = f"""
        UPDATE {table_name}
        SET {column_name} = REPLACE({column_name}, ?, ?)
        """
        cursor.execute(query, (linebreak, ','))
    
    # Commit the changes and close the connection
    conn.commit()
    cursor.close()
    conn.close()

In [22]:
replace_linebreaks_with_comma("bibliography", "Language")

In [23]:
get_unique_values("bibliography", "Language")

['Farsi',
 None,
 'Arabic',
 'Farsi,Tajik',
 'Russian',
 'English',
 'Turkic',
 'Turkic,Tatar',
 'French',
 'French,Russian',
 'Farsi,Turkic,Russian,Tajik,Uzbek',
 'Farsi,Judeo-Persian',
 'Turkic,Uyghur',
 'Judeo-Persian,Farsi',
 'Farsi,Turkic',
 'Farsi,Dari',
 'Uzbek',
 'Russian,Farsi',
 'Urdu',
 'Farsi,Russian',
 'Turkic,Tatar,Arabic',
 'Tajik',
 'Turkic,Azeri',
 'Arabic,Turkic',
 'Russian,Turkic',
 'Uyghur',
 'Arabic,Farsi',
 'Turkic,Uzbek']

### Function: `add_check_constraint`

This function adds a `CHECK` constraint to a specified column in an existing SQLite table, ensuring the column can only contain specified values.

#### Steps:

1. **Initialize Cursor**: Create a cursor from the global SQLite connection to execute SQL commands.

2. **Prepare Allowed Values**: Convert the list of allowed values into a SQL-compatible string using the `IN` operator.

3. **Retrieve Table Schema**: Use `PRAGMA table_info` to get the current table schema.

4. **Construct New Schema**: Build a new table schema, adding a `CHECK` constraint to the specified column.

5. **Create New Table**: Create a new table with the updated schema.

6. **Copy Data**: Transfer data from the old table to the new table.

7. **Replace Old Table**: Drop the old table and rename the new table to the original name.

8. **Commit Changes**: Commit the changes and close the cursor.

#### Example Usage

```python
# Example usage of the function
table_name = 'bibliography'  # Name of the table to modify
column_name = 'Language'     # Name of the column to apply the constraint to
allowed_values = ['Russian', 'Farsi', 'English', 'French']  # List of allowed values

add_check_constraint(table_name, column_name, allowed_values)
```

- **`table_name`**: The name of the table you want to modify, e.g., `'bibliography'`.
- **`column_name`**: The name of the column to add the constraint to, e.g., `'Language'`.
- **`allowed_values`**: A list of allowed string values, enclosed in square brackets, e.g., `['Russian', 'Farsi', 'English', 'French']`.

In [13]:
import sqlite3

# Assume conn is a global variable or defined in the same scope
conn = sqlite3.connect('path_to_your_database.db')  # Example connection setup

def add_check_constraint(table_name, column_name, allowed_values):
    cursor = conn.cursor()
    
    # Convert the list of allowed values into a string suitable for SQL
    allowed_values_str = ', '.join(f"'{value}'" for value in allowed_values)
    
    # Get the existing table schema
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns_info = cursor.fetchall()
    
    # Create a new table schema with the CHECK constraint
    new_table_columns = []
    for column in columns_info:
        col_name = column[1]
        col_type = column[2]
        
        # Check for existing constraints
        existing_constraints = ""
        if col_name == column_name:
            # Retrieve existing CHECK constraints (if any)
            cursor.execute(f"PRAGMA table_info({table_name});")
            for col in cursor.fetchall():
                if col[1] == column_name and col[5]:  # col[5] is the default value, which may include constraints
                    existing_constraints = col[5]
            
            # Combine existing constraints with the new one
            if existing_constraints:
                new_constraint = f"{existing_constraints} AND {col_name} IN ({allowed_values_str})"
            else:
                new_constraint = f"{col_name} IN ({allowed_values_str})"
            
            new_table_columns.append(f"{col_name} {col_type} CHECK({new_constraint})")
        else:
            new_table_columns.append(f"{col_name} {col_type}")
    
    new_table_schema = ', '.join(new_table_columns)
    
    # Create a new table with the same structure and the added CHECK constraint
    new_table_name = f"{table_name}_new"
    cursor.execute(f"CREATE TABLE {new_table_name} ({new_table_schema});")
    
    # Copy data from the old table to the new table
    column_names = ', '.join(column[1] for column in columns_info)
    cursor.execute(f"INSERT INTO {new_table_name} ({column_names}) SELECT {column_names} FROM {table_name};")
    
    # Drop the old table
    cursor.execute(f"DROP TABLE {table_name};")
    
    # Rename the new table to the original table name
    cursor.execute(f"ALTER TABLE {new_table_name} RENAME TO {table_name};")
    
    # Commit the changes
    conn.commit()
    cursor.close()

# Example usage
table_name = 'bibliography'
column_name = 'Language'
allowed_values = ['Russian', 'Farsi', 'English', 'French']

add_check_constraint(table_name, column_name, allowed_values)

# Don't forget to close the connection when done
conn.close()

OperationalError: near ")": syntax error

In [21]:


# Function to enable regex in SQLite
def regex_search(pattern, string):
    # Check if the string is valid
    if not isinstance(string, str):
        return False
    try:
        return re.search(pattern, string) is not None
    except Exception as e:
        print(f"Regex error: {e}")
        return False

# Register the regex function with SQLite
def register_regex(conn):
    conn.create_function("REGEXP", 2, regex_search)

def word_search(search_term):
    # Establish a connection to the database
    conn = sqlite3.connect(database_path)
    register_regex(conn)  # Register the regex function
    cursor = conn.cursor()
    
    try:
        # SQL query to search through multiple columns in the lexicon table and join on definitions
        query = """
        SELECT l.UID, l.Term, l.Translation, l.Emic_Term, l.Colonial_Term, l.Transliteration, d.Definition
        FROM lexicon l
        JOIN definitions d ON l.UID = d.Lexicon_ID
        WHERE l.Term REGEXP ? OR l.Translation REGEXP ? OR l.Emic_Term REGEXP ? OR l.Colonial_Term REGEXP ? OR l.Transliteration REGEXP ?;
        """
        
        # Execute the query with the search term
        cursor.execute(query, (search_term, search_term, search_term, search_term, search_term))
        
        # Fetch all results
        results = cursor.fetchall()
        
        # Create a DataFrame with labeled columns
        columns = ['UID', 'Term', 'Translation', 'Emic_Term', 'Colonial_Term', 'Transliteration', 'Definition']
        df = pd.DataFrame(results, columns=columns)
        
        # Set display options for long text
        pd.set_option('display.max_colwidth', None)  # Show full content of the Definition column
        
        # Initialize Related_Terms column
        df['Related_Terms'] = ''
        
        # Check for related terms
        for index, row in df.iterrows():
            uid = row['UID']
            # Query to find related terms
            related_query = """
            SELECT l.Term
            FROM related_terms rt
            JOIN lexicon l ON rt.Child_ID = l.UID
            WHERE rt.Parent_ID = ?;
            """
            cursor.execute(related_query, (uid,))
            related_terms = cursor.fetchall()
            # Concatenate related terms into a single string
            related_terms_list = [term[0] for term in related_terms]
            df.at[index, 'Related_Terms'] = ', '.join(related_terms_list)
        
    except Exception as e:
        print(f"Error during query execution: {e}")
    finally:
        # Close the cursor and connection
        cursor.close()
        conn.close()
    
    return df

# Example usage
# search_results_df = word_search('your_regex_pattern')
# print(search_results_df)

In [6]:
word_search ('باج')

# next step: optional second argument that filters based on tags

,UID,Term,Translation,Emic_Term,Colonial_Term,Transliteration,Definition,Related_Terms
0,54,baj,None,باج,,None,ماليات گمركي كه بر بازرگانان و قبيلههاي چادرنشين روسي بسته ميشد,


In [23]:


def location_search(search_term):
    # Establish a connection to the database
    conn = sqlite3.connect(database_path)
    register_regex(conn)  # Register the regex function
    cursor = conn.cursor()
    
    try:
        # SQL query to search through multiple columns in the gazetteer table
        query = """
        SELECT UID, Nickname, Location_Name_Arabic, Location_Name_Colonial, Location_Name_Latin
        FROM gazetteer
        WHERE Nickname REGEXP ? OR Location_Name_Arabic REGEXP ? OR Location_Name_Colonial REGEXP ? OR Location_Name_Latin REGEXP ?;
        """
        
        # Execute the query with the search term
        cursor.execute(query, (search_term, search_term, search_term, search_term))
        
        # Fetch all matching records
        matching_records = cursor.fetchall()
        
        # If no matches found, return an empty DataFrame
        if not matching_records:
            return pd.DataFrame()  # Return an empty DataFrame if no matches
        
        # Extract UIDs from the results
        uid_list = [record[0] for record in matching_records]
        
        # SQL query to join with location_attributes based on Location_ID
        attributes_query = """
        SELECT la.*, g.Nickname, g.Location_Name_Arabic, g.Location_Name_Colonial, g.Location_Name_Latin
        FROM location_attributes la
        JOIN gazetteer g ON la.Location_ID = g.UID
        WHERE g.UID IN ({});
        """.format(','.join('?' * len(uid_list)))  # Create placeholders for the UIDs
        
        # Execute the query with the list of UIDs
        cursor.execute(attributes_query, uid_list)
        
        # Fetch all results
        results = cursor.fetchall()
        
        # Create a DataFrame with all columns from location_attributes and additional columns from gazetteer
        columns = [description[0] for description in cursor.description]  # Get column names
        df = pd.DataFrame(results, columns=columns)
        
    except Exception as e:
        print(f"Error during query execution: {e}")
        df = pd.DataFrame()  # Return an empty DataFrame on error
    finally:
        # Close the cursor and connection
        cursor.close()
        conn.close()
    
    return df

# Example usage
# location_results_df = location_search('your_regex_pattern')
# print(location_results_df)

In [19]:
location_search ('sh.hr')

# next steps: output information from location_hierarchies as well

NameError: name 'register_regex' is not defined